In [1]:
import os
os.getcwd()
# os.chdir(default_path)

'C:\\Users\\hanjh\\Desktop\\Kaggle project\\AB Testing'

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import mglearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, ParameterGrid, RepeatedKFold, StratifiedKFold, RepeatedStratifiedKFold, StratifiedKFold
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
from sklearn.feature_selection import RFECV, VarianceThreshold, SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, mean_squared_error, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelBinarizer, label_binarize
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.pipeline import Pipeline as Pipeline_imb
from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVC, SVC, SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from summarytools import dfSummary
from sklearn.metrics import roc_auc_score, RocCurveDisplay, precision_recall_curve
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import colorama
from colorama import Fore, Style  # maakes strings colored
# !pip3 install termcolor
from termcolor import colored

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings(action='ignore')

In [9]:
AB_df = pd.read_csv("cookie_cats.csv")

## A/B Testing

### Overview
In A/B testing, choosing the right statistical test depends on the nature of your data, especially its distribution and variance.

#### 1. Check for Normality
First, determine if your data is normally distributed. You can use statistical tests like the Shapiro-Wilk test or visual tools like histograms or Q-Q plots to assess this. If the data is normally distributed, you can proceed with parametric tests like a t-test.

#### 2. Parametric Test (t-test)
Use the t-test (independent t-test) if:
- Your data is normally distributed.
- The variances between the two groups are equal (using Levene's test for equality of variances).
- The two samples are independent of each other.

##### Types of t-tests:
- **Independent t-test**: Compare means between two independent groups (e.g., A and B).
- **Paired t-test**: If you have repeated measurements from the same subjects (e.g., before and after tests on the same users).


##### 2-1) Levene's Test and Welch's t-test

In a parametric t-test, the Levene's test for equality of variances is performed to check if the two groups being compared have equal variances (homoscedasticity). This is important because the standard independent t-test assumes that the variances of the two groups are equal. 

##### Why Check for Equal Variances?
##### Assumption of the Standard t-test
The standard (Student's) t-test assumes that the two groups you're comparing have the same variance. 
If the variances of the two groups are not equal (heteroscedasticity), using the standard t-test can lead to inaccurate results. 

##### Levene's Test
The Levene’s test checks whether the variances between the two groups are equal. 
If the test returns a p-value less than your significance level (e.g., 0.05), you can conclude that the variances are significantly different, and the assumption of equal variances is violated.

##### What Happens if Variances Are Unequal?
If Levene's test indicates that the variances are not equal, two options:

1. **Use Welch’s t-test (an alternative to the standard t-test):**
   - Welch's t-test is a version of the t-test that does not assume equal variances. It adjusts the degrees of freedom based on the variances of the two groups.
   - This test is more reliable when the variances of the two groups are unequal and should be used whenever Levene’s test indicates unequal variances.

2. **Proceed with caution using the standard t-test:**
   - If you continue using the standard t-test despite unequal variances, your results may not be reliable, especially if the sample sizes are also unequal. The Welch’s t-test is always a safer choice in such cases.

---

In [29]:
## Example: Levene's Test and Welch’s t-test in Python
import numpy as np
from scipy import stats

# Sample data: Group A and Group B
group_a = np.random.normal(loc=10, scale=2, size=30)  # Mean 10, SD 2
group_b = np.random.normal(loc=12, scale=5, size=30)  # Mean 12, SD 5

# Levene's Test for equality of variances
stat, p_value = stats.levene(group_a, group_b)

print(f"Levene’s test statistic: {stat}")
print(f"P-value from Levene's test: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: Variances are not equal.")
    # Perform Welch’s t-test (does not assume equal variances)
    t_stat, p_val_welch = stats.ttest_ind(group_a, group_b, equal_var=False)
    print(f"Welch's t-test result: t-statistic = {t_stat}, p-value = {p_val_welch}")
else:
    print("Fail to reject the null hypothesis: Variances are equal.")
    # Perform standard t-test (assuming equal variances)
    t_stat, p_val_ttest = stats.ttest_ind(group_a, group_b, equal_var=True)
    print(f"Standard t-test result: t-statistic = {t_stat}, p-value = {p_val_ttest}")

Levene’s test statistic: 12.346909938460435
P-value from Levene's test: 0.0008636348956642888
Reject the null hypothesis: Variances are not equal.
Welch's t-test result: t-statistic = -2.6681938551004123, p-value = 0.0106543969626268


#### 3. Non-Parametric Test (Mann-Whitney U test)
Use the Mann-Whitney U test (also called the Wilcoxon rank-sum test) if:
- The data is not normally distributed.
- The samples are independent of each other.
- It tests for differences in the distribution between two groups.

This test is a non-parametric alternative to the t-test when assumptions of normality are violated.

#### 4. Chi-Square Test
If your data is categorical (e.g., conversion rate: success/fail, clicks/no clicks), you might use a chi-square test to test for differences in proportions between the two groups.

#### 5. Two-Proportion Z-test
If you're comparing proportions between two groups (e.g., comparing conversion rates between group A and B), and both samples are large enough, you can use the two-proportion Z-test.

##### Summary of Key Tests:
- **Independent t-test**: Normally distributed data, comparing means.
- **Paired t-test**: Repeated measures from the same group.
- **Mann-Whitney U test**: Non-normally distributed data, comparing distributions.
- **Chi-square test**: Categorical data.
- **Two-proportion Z-test**: Large sample sizes, comparing proportions.

#### How to Choose:
- Start by checking if your data is normally distributed. If yes, go for the t-test.
- If not, use the Mann-Whitney U test.
- For categorical data, consider the chi-square test or Z-test for proportions.
---

#### Chi-Square Test Example

##### Use Case: Comparing Categorical Data
You would use the Chi-Square test when comparing categorical data between two groups. For example, you want to see if there's a significant difference in the click-through rates (CTR) between two different versions of a webpage (A and B).

##### Scenario:
- Group A (Control) has 200 users, 50 of whom clicked on an ad.
- Group B (Variation) has 180 users, 70 of whom clicked on an ad.

In [66]:
import numpy as np
import scipy.stats as stats

# Contingency table
data = np.array([[50, 150],  # Group A: 50 clicked, 150 did not click
                 [70, 110]]) # Group B: 70 clicked, 110 did not click

# Chi-Square Test
chi2, p, dof, expected = stats.chi2_contingency(data)

# Results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")
print(f"Expected Frequencies: {expected}")

# Interpretation
if p < 0.05:
    print("Reject the null hypothesis: There is a significant difference in click-through rates.")
else:
    print("Fail to reject the null hypothesis: No significant difference in click-through rates.")

Chi-Square Statistic: 7.82738425925926
P-value: 0.005146051829425319
Degrees of Freedom: 1
Expected Frequencies: [[ 63.15789474 136.84210526]
 [ 56.84210526 123.15789474]]
Reject the null hypothesis: There is a significant difference in click-through rates.


#### Two-Proportion Z-Test example
Use the Two-Proportion Z-test when comparing proportions between two independent groups. This test is common in A/B testing for conversion rates or success rates.

##### Scenario:

Group A has 1000 users, 100 of whom converted (10% conversion rate).<br/>
Group B has 1200 users, 150 of whom converted (12.5% conversion rate).<br/>
You want to test if the conversion rate in Group B is significantly different from Group A.

In [73]:
import statsmodels.api as sm

# Data: Successes and Total Observations for both groups
success_a = 100   # Conversions in Group A
nobs_a = 1000     # Total users in Group A
success_b = 150   # Conversions in Group B
nobs_b = 1200     # Total users in Group B

# Perform Two-Proportion Z-test
count = np.array([success_a, success_b])
nobs = np.array([nobs_a, nobs_b])

zstat, pval = sm.stats.proportions_ztest(count, nobs)

# Results
print(f"Z-Statistic: {zstat}")
print(f"P-value: {pval}")

if pval < 0.05:
    print("Reject the null hypothesis: The conversion rates are significantly different.")
else:
    print("Fail to reject the null hypothesis: No significant difference in conversion rates.")

Z-Statistic: -1.839732422015599
P-value: 0.06580753108398388
Fail to reject the null hypothesis: No significant difference in conversion rates.


## Implement A/B Testing 

In [75]:
AB_df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [77]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.DataFrame(data)

# Split the data into two groups based on 'version'
gate_30 = AB_df[AB_df['version'] == 'gate_30']['sum_gamerounds']
gate_40 = AB_df[AB_df['version'] == 'gate_40']['sum_gamerounds']

### 1. Check for Normality using Shapiro-Wilk Test
print("Normality Test (Shapiro-Wilk Test)")

# Shapiro test for gate_30
shapiro_gate_30 = stats.shapiro(gate_30)
print(f"gate_30: W-statistic={shapiro_gate_30[0]}, p-value={shapiro_gate_30[1]}")

# Shapiro test for gate_40
shapiro_gate_40 = stats.shapiro(gate_40)
print(f"gate_40: W-statistic={shapiro_gate_40[0]}, p-value={shapiro_gate_40[1]}")

# Interpretation of Shapiro-Wilk Test
if shapiro_gate_30[1] < 0.05 or shapiro_gate_40[1] < 0.05:
    print("At least one group is not normally distributed. Consider using a non-parametric test.")
else:
    print("Both groups appear normally distributed.")

Normality Test (Shapiro-Wilk Test)
gate_30: W-statistic=0.08805022308043553, p-value=2.146395844442685e-157
gate_40: W-statistic=0.482561004889815, p-value=3.3446548187520663e-140
At least one group is not normally distributed. Consider using a non-parametric test.


In [44]:
### 2. Check for Equal Variances using Levene’s Test
print("\nVariance Test (Levene's Test)")
stat, p_value_levene = stats.levene(gate_30, gate_40)
print(f"Levene’s test statistic: {stat}, p-value: {p_value_levene}")

if p_value_levene < 0.05:
    print("Variances are not equal (heteroscedasticity). Use Welch's t-test or non-parametric tests.")
else:
    print("Variances are equal (homoscedasticity).")


Variance Test (Levene's Test)
Levene’s test statistic: 0.5292002638313259, p-value: 0.46694516772623273
Variances are equal (homoscedasticity).


In [79]:
### 3. Perform T-test or Welch's T-test
# Check if normality and variance assumptions are met
if p_value_levene > 0.05:
        # Use standard t-test (equal variances)
        t_stat, p_val_ttest = stats.ttest_ind(gate_30, gate_40, equal_var=True)
        print(f"Standard t-test result: t-statistic={t_stat}, p-value={p_val_ttest}")
else:
    # Use Welch's t-test (unequal variances)
    t_stat, p_val_ttest = stats.ttest_ind(gate_30, gate_40, equal_var=False)
    print(f"Welch's t-test result: t-statistic={t_stat}, p-value={p_val_ttest}")

Standard t-test result: t-statistic=0.8910426211362967, p-value=0.37290868247405207


In [53]:
### 4. If Normality Violated, Perform Non-Parametric Mann-Whitney U Test

# Non-parametric Mann-Whitney U test
u_stat, p_val_mw = stats.mannwhitneyu(gate_30, gate_40, alternative='two-sided')
print(f"Mann-Whitney U Test result: U-statistic={u_stat}, p-value={p_val_mw}")


Mann-Whitney U Test result: U-statistic=1024331250.5, p-value=0.05020880772044255


If the p-value from the Mann-Whitney U test is greater than 0.05, it means that there is no statistically significant difference between the distributions of the sum_gamerounds for the two versions (Gate_30 and Gate_40).

**Here's how you can interpret the result:<br/>**
- Null Hypothesis (H₀): The distributions of sum_gamerounds for Gate_30 and Gate_40 are the same. <br/>
- Alternative Hypothesis (H₁): The distributions of sum_gamerounds for Gate_30 and Gate_40 are different. <br/>
Since the p-value is greater than 0.05, you fail to reject the null hypothesis, meaning that there is no strong evidence to suggest that the two versions (Gate_30 and Gate_40) result in different distributions of sum_gamerounds.

**Conclusion:**
The Mann-Whitney U test suggests that there is no significant difference in the number of game rounds played between users in the Gate_30 and Gate_40 groups.

**Practical Interpretation:**
This result implies that changing the version from Gate_30 to Gate_40 likely does not have a notable impact on the total number of game rounds played by users. Based on this test, you might conclude that the change in the version (from Gate_30 to Gate_40) does not affect user engagement, as measured by sum_gamerounds.